<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/multi-turn-chat/multi-turn-conversation.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Navigator Data Designer SDK: Synthetic Conversational Data with Person Details

This notebook demonstrates how to use the Gretel Navigator SDK to build a synthetic data generation pipeline step-by-step. We will create multi-turn user-assistant dialogues tailored for fine-tuning language models, enhanced with realistic person details. These synthetic dialogues can then be used as domain-specific training data to improve model performance in targeted scenarios.

These datasets could be used for developing and enhancing conversational AI applications, including customer support chatbots, virtual assistants, and interactive learning systems.

In [13]:
%%capture
# Install the latest version of Gretel client and dependencies
%pip install -U gretel_client 

In [14]:
from typing import Literal
from pydantic import BaseModel, Field # type: ignore
from gretel_client.navigator_client import Gretel # type: ignore

## ⚙️ Data Designer Configuration with the SDK

Instead of relying on a single YAML configuration file, here we build up our pipeline interactively. This provides granular control over how we guide LLMs to produce realistic, domain-specific conversations. By adjusting prompts, seed data, and instructions, we can quickly iterate and refine our data generation process.

### 📚 Choosing the Model Suite
Specify the `model_suite` to determine which models and associated licenses are used during data generation.
For example, use `apache-2.0` for open-source-friendly licensing or `llama-3.x` or `amazon-nova` for advanced proprietary models.
Select the suite based on compliance and licensing requirements relevant to your use case.

In [15]:
# Available model suites: apache-2.0, llama-3.x
model_suite = "apache-2.0"

### ✍️ Setting Special System Instructions

Provide system-wide instructions for the underlying LLMs to guide the data generation process. These instructions influence all generated dialogues, ensuring consistency, quality, and adherence to desired rules. The instructions specify guidelines for factual accuracy, contextual relevance, and tone.


### 🚀 Initialize Gretel Navigator Data Designer

Instantiate the Gretel client and create a new data designer with the chosen model suite and special system instructions.

In [16]:
gretel = Gretel(api_key="prompt", endpoint='https://api.dev.gretel.ai')

# Initialize the new Data Designer
aidd = gretel.data_designer.new(    
    model_suite=model_suite
)

Found cached Gretel credentials
Logged in as kirit.thadaka@gretel.ai ✅
Gretel client configured to use project: proj_2uY0cfM0kjiegpyEZvCHNKZYxGf


### Define Pydantic Models for Structured Outputs

You can use Pydantic to define a structure for the messages that are produced by Data Designer

In [17]:
class Message(BaseModel):
    """A single message turn in the conversation."""
    role: Literal["user", "assistant"] = Field(..., description="Which role is writing the message.")
    content: str = Field(..., description="Message contents.")


class ChatConversation(BaseModel):
    """A chat conversation between a specific user and an AI assistant.
    * All conversations are initiated by the user role.
    * The assistant role always responds to the user message.
    * Turns alternate between user and assistant roles.
    * The last message is always from the assistant role.
    * Message content can be long or short.
    * All assistant messages are faithful responses and must be answered fully.
    """
    conversation: list[Message] = Field(..., description="List of all messages in the conversation.")


class UserToxicityScore(BaseModel):
    """Output format for user toxicity assessment.

    Toxicity Scores:
    None: No toxicity detected in user messages.
    Mild: Slightly rude or sarcastic but not hateful or harmful.
    Moderate: Some disrespectful or harassing language.
    Severe: Overt hate, harassment, or harmful content.
    """
    reasons: list[str] = Field(..., description="Reasoning for user toxicity score.")
    score: Literal["None", "Mild", "Moderate", "Severe"] = Field(..., description="Level of toxicity observed in the user role responses.")

### 🌱 Adding Categorical Seed Columns

Define categorical seed columns that set the context for the generated dialogues. Domain, topic, complexity, conversation length, and user mood will influence the generated conversations.

In [18]:
# Add domain column with subcategories for topics
aidd.add_column(
    name="domain",
    type="category",
    params={
        "values": ["Tech Support", "Personal Finances", "Educational Guidance"],
        "num_new_values_to_generate": 5
    }
)

# Add topic subcategory
aidd.add_column(
    name="topic",
    type="subcategory",
    params={
        "category": "domain",
        "values": {
            "Tech Support": [
                "Troubleshooting a Laptop",
                "Setting Up a Home Wi-Fi Network",
                "Installing Software Updates"
            ],
            "Personal Finances": [
                "Budgeting Advice",
                "Understanding Taxes",
                "Investment Strategies"
            ],
            "Educational Guidance": [
                "Choosing a College Major",
                "Effective Studying Techniques",
                "Learning a New Language"
            ]
        },
        "num_new_values_to_generate": 2
    }
)

# Add complexity column
aidd.add_column(
    name="complexity",
    type="category",
    params={
        "values": ["Basic", "Intermediate", "Advanced"]
    }
)

# Add conversation length column
aidd.add_column(
    name="conversation_length",
    type="category",
    params={
        "values": [2, 4, 6, 8]
    }
)

# Add user mood column
aidd.add_column(
    name="user_mood",
    type="category",
    params={
        "values": ["happy", "silly", "sarcastic", "combative", "disappointed", "toxic"]
    }
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: [
        "domain",
        "topic",
        "complexity",
        "conversation_length",
        "user_mood"
    ]
)

### ✨ Adding Generated Data Columns
Now define the columns that the model will generate. These prompts instruct the LLM to produce the actual conversation: a system prompt to guide how the AI assistant engages in the conversation with the user, the conversation, and finally, we generate a toxicity_label to assess user toxicity over the entire conversation.

#### 💬🤖 AI Assistant system prompt and conversation

We generate a system prompt to base the AI assistant and then generate the entire conversation.

In [19]:
# Generate assistant system prompt
aidd.add_column(
    name="assistant_system_prompt",
    type="llm-text",
    system_prompt="Keep this to a maximum of two sentences.",
    prompt="Write a reasonable system prompt for a helpful AI assistant with expertise in {{domain}} and {{topic}}. The AI assistant must not engage in harmful behaviors."
)

# Generate the user's task
aidd.add_column(
    name="user_task",
    type="llm-text",
    system_prompt="The task should be clear, focused on a single goal, and at most two sentences. Focus only on the task and don't provide only the task information.",
    prompt="Define a simple task related to {{topic}} of {{complexity}} complexity for the user."
)


# Generate the conversation
aidd.add_column(
    name="conversation",
    type="llm-structured",
    prompt=(    
        "<task>\n{{user_task}}\n</task>\n\n"

        "<system_prompt>{{assistant_system_prompt}}</system_prompt>\n\n"

        "Generate a conversation between a user and an AI assistant with <system_prompt> about <task>.\n"
        "User is asking the assistant for advice and is in a {{user_mood}} mood.\n"
        "The conversation must be {{conversation_length}} messages in length.\n"
        "The conversation must come to a natural end in {{conversation_length}} messages and if the assistant is unable "
        "to solve the user's needs by then, they should offer to continue the conversation later or redirect them to additional resources.\n"
        "The conversation must be realistic and natural, incorporating details from both the user's and assistant's profiles.\n"
        "The AI assistant role has the system prompt defined in <system_prompt>. All of its responses must conform to <system_prompt>:\n"
    ),
    output_format=ChatConversation
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: [
        "domain",
        "topic",
        "complexity",
        "conversation_length",
        "user_mood"
    ]
    llm_gen_columns: ['assistant_system_prompt', 'user_task', 'conversation']
)

#### 🔍⚖️ Assessment Column: Toxicity Label
For the final step, we add a toxicity assessment column to classify the overall toxicity level of the user's messages across the entire conversation.

In [20]:
# # Toxicity Label for the entire conversation

from gretel_client.data_designer.params import Rubric

# Define Toxicity Rubric
toxicity_rubric = Rubric(
    name="Toxicity",
    description="Evaluation of harmful, harassing, or hateful language in conversations",
    scoring={
        "4": "No toxicity detected; conversation is respectful and appropriate.",
        "3": "Mild toxicity; slightly rude or sarcastic but not hateful or harmful.",
        "2": "Moderate toxicity; contains disrespectful or harassing language.",
        "1": "Severe toxicity; contains overt hate, harassment, or harmful content.",
        "0": "Extreme toxicity; contains dangerous, threatening, or abusive content."
    }
)

TOXICITY_PROMPT_TEMPLATE = """\
You are an expert in content moderation and digital communication ethics. You are evaluating the toxicity level in user messages.
Take a deep breath and carefully analyze the conversation provided. Use the rubric to score the level of toxicity in the USER'S messages only.

Consider:
- Focus primarily on harassment, hate, or harmful language
- Evaluate the full context of the conversation
- Be objective and consistent in your assessment
- If any single message exhibits toxicity, it should influence the final score

## CONVERSATION
{{ conversation }}
"""

aidd.add_column(
    name="toxicity_evaluation",
    type='llm-judge',
    prompt=TOXICITY_PROMPT_TEMPLATE,
    rubrics=[toxicity_rubric]
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: [
        "domain",
        "topic",
        "complexity",
        "conversation_length",
        "user_mood"
    ]
    llm_gen_columns: ['assistant_system_prompt', 'user_task', 'conversation']
    llm_judge_columns: ['toxicity_evaluation']
)

## 👀 Generating a dataset preview

- Preview mode allows you to quickly iterate on your data design.

- Each preview generation call creates 10 records for inspection, helping you verify prompts and instructions before running a larger batch job.

In [21]:
# Generate a preview
preview = aidd.preview(verbose_logging=True)

[09:27:24] [INFO] 🚀 Generating preview
[09:27:24] [INFO] ⛓️ Representing generation steps as a Directed Acyclic Graph
[09:27:24] [INFO]   |-- 🔗 `conversation` depends on `user_task`
[09:27:24] [INFO]   |-- 🔗 `conversation` depends on `assistant_system_prompt`
[09:27:24] [INFO]   |-- 🔗 `toxicity_evaluation` depends on `conversation`
[09:27:26] [INFO] 🦜 Step 1: Generate columns using samplers
[09:27:26] [INFO]   |-- 🎲 Using numerical samplers to generate 10 records across 5 columns
[09:27:27] [INFO] 🦜 Step 2: Generate column from template v2
[09:27:27] [INFO]   |-- 📝 Preparing template to generate data column `assistant_system_prompt`
[09:27:27] [INFO]   |   |-- model_alias: ModelAlias.TEXT
[09:27:29] [INFO]   |-- Generation summary for field: assistant_system_prompt
[09:27:29] [INFO]   |-- 	Total inference requests: 10
[09:27:29] [INFO]   |-- 	Successful requests: 10
[09:27:29] [INFO]   |-- Model usage: [{"model": "gretel/mistralai/Mistral-Small-24B-Instruct-2501", "prompt_tokens": 461,

## 🔎 Easily inspect individual records

- Run the cell below to display individual records for inspection.

- Run the cell multiple times to cycle through the 10 preview records.

- Alternatively, you can pass the `index` argument to display a specific record.

In [22]:
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                    ┃ Value                                                                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ domain                  │ Tech Support                                                                          │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ topic                   │ Installing Software Updates                                                           │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ complexity              │ Basic                                                                                 │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ conversation_length     │ 4                                                                                     │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ user_mood               │ silly                                                                                 │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ assistant_system_prompt │ You are a helpful AI assistant specializing in tech support and software updates.     │
│                         │ Your primary goal is to assist users with troubleshooting technical issues, guiding   │
│                         │ them through software updates, and providing clear, step-by-step instructions while   │
│                         │ ensuring all interactions are safe, respectful, and non-harmful.                      │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ user_task               │ Check if there are any available software updates for your operating system.          │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ conversation            │ {                                                                                     │
│                         │     'conversation': [                                                                 │
│                         │         {                                                                             │
│                         │             'role': 'user',                                                           │
│                         │             'content': "Hey there, tech guru! I've been feeling a bit outdated        │
│                         │ myself, so I thought maybe my computer needs an update too. Any chance you can help   │
│                         │ me check for software updates?"                                                       │
│                         │         },                                                                            │
│                         │         {                                                                             │
│                         │             'role': 'assistant',                                                      │
│                         │             'content': "Of course! I'd be happy to help. First, let's figure out what │
│                         │ operating system you're using. Are you on Windows, Mac, or something else?"           │
│                         │         },                                                                            │
│                         │         {                   

## 🤔 Like what you see?

Submit a batch workflow!

In [23]:
# # # Submit batch job
# workflow_run = aidd.create(
#     num_records=100,
#     name="multi_turn_conversation_with_person_details",
#     wait_for_completion=True
# )
# print("\nGenerated dataset shape:", workflow_run.dataset.df.shape)

By following these steps and leveraging the interactivity of the SDK, you can refine prompts, generate realistic dialogues with detailed personas, and ensure the resulting dataset is high-quality, non-toxic, and aligned with your domain-specific requirements.

In [24]:
# # Inspect first 10 records of the generated dataset
# workflow_run.dataset.df.head(10)